In [1]:
pip install ktrain

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import ktrain
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
issues = pd.read_csv("issues.csv")

In [11]:
len(issues["sample"].unique())

68

In [7]:
issues["title_body"] = issues["title"] + issues["body"]

In [8]:
issues['title_body'] = issues['title_body'].str.lower()

In [9]:
def remove_unused_chars():
  remove = ["-", ">", "<", "[", "]", "`", "´", "#", "=", "{", "}", ",", ".", "!", "?", ";", "/", "+", "(", ")", ":", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", '\"', "'", "^", "\\", "&", "%", "*", "@", '</s>', "|", "\n", "\t"]
  for r in remove:
    issues["title_body"] = issues["title_body"].str.replace(r, "")

In [10]:
remove_unused_chars()

In [11]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
wordnet_lemmatizer = WordNetLemmatizer()
nrows = len(issues)
lemmatized_text_list = []

for row in range(0, nrows):
    
    lemmatized_list = []
    
    text = issues.loc[row]["title_body"]
    text_words = str(text).split(" ")

    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    lemmatized_text = " ".join(lemmatized_list)
    
    lemmatized_text_list.append(lemmatized_text)

In [13]:
issues["title_body"] = lemmatized_text_list

In [14]:
remove_unused_chars()

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
stop_words = list(stopwords.words('english'))
for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    issues["title_body"] = issues["title_body"].str.replace(regex_stopword, '')

In [17]:
remove_unused_chars()

In [18]:
(x_train, y_train) , (x_test, y_test), preproc = ktrain.text.texts_from_df(issues, text_column="title_body", preprocess_mode="bert", label_columns=["label"], val_pct=0.3)

['non-question', 'question']
     non-question  question
60            1.0       0.0
219           1.0       0.0
46            1.0       0.0
298           0.0       1.0
133           1.0       0.0
['non-question', 'question']
     non-question  question
200           0.0       1.0
187           1.0       0.0
295           0.0       1.0
118           1.0       0.0
63            1.0       0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [19]:
classifier = ktrain.text.text_classifier(name="bert", train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 400
done.


In [20]:
learner = ktrain.get_learner(classifier, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=4)

In [20]:
learner.lr_find()

simulating training for different learning rates... this may take a few moments...


/opt/conda/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
/opt/conda/lib/python3.8/site-packages/ktrain/core.py:566: UserWarning: max_epochs is being set to 5 since steps per epoch is small. If you wish to estimate LR using more epochs, set max_epochs manually.
  warnings.warn('max_epochs is being set to 5 since steps per epoch is small. ' +\


Epoch 1/5


In [21]:
learner.lr_plot()

ValueError: Please call lr_find first.

In [ ]:
learner.fit_onecycle(2e-3, 3)



begin training using onecycle policy with max lr of 0.002...
Epoch 1/3
 3/57 [>.............................] - ETA: 9:41 - loss: 0.5870 - accuracy: 0.9167 

In [ ]:
learner.evaluate((x_test, y_test))

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()